In [20]:
from DataLoader import *
from TransCGAN_model import * 

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils import data

from minerva.data.data_modules.har_rodrigues_24 import HARDataModuleCPC


In [11]:
train_set = mitbih_train(filename='/workspaces/container-workspace/tts-cgan/mitbih/mitbih_train.csv')
train_loader = data.DataLoader(train_set, batch_size=64, num_workers=10, shuffle=True)


X_train shape is (100000, 1, 1, 187)
y_train shape is (100000,)
The dataset including 20000 class 0, 20000 class 1, 20000 class 2, 20000 class 3, 20000 class 4


In [19]:
batch = next(iter(train_loader))
batch[1]

tensor([2., 4., 4., 2., 4., 0., 4., 3., 3., 4., 0., 0., 3., 3., 0., 0., 0., 4.,
        0., 3., 0., 2., 1., 1., 2., 2., 0., 1., 3., 1., 2., 0., 0., 4., 4., 3.,
        1., 1., 2., 3., 0., 3., 4., 4., 0., 1., 1., 4., 3., 1., 0., 0., 0., 2.,
        1., 4., 1., 4., 0., 1., 4., 0., 0., 0.], dtype=torch.float64)

In [27]:
data_path = '/workspaces/container-workspace/CPC_daghar_data'
datasets = ['kuhar', 'motion', 'rw-thigh', 'rw-waist', 'uci', 'wisdm']
datasets_path = [os.path.join(data_path ,current_dataset) for current_dataset in datasets]

data_module = HARDataModuleCPC(
    data_path=datasets_path,
    input_size=6,
    window=60,
    overlap=60,
    batch_size=64,
    label='standard activity code')

data_module.setup('fit')
train_loader = data_module.train_dataloader()

In [ ]:
def reshape_daghar_loader(train_loader):
    """
    Recebe um DataLoader e retorna um novo DataLoader
    onde os tensores do batch têm shape (B, 6, 1, 60),
    convertendo automaticamente os dados de numpy para tensor.
    """

    def collate_fn_with_unsqueeze(batch):
        xs, ys = zip(*batch)  # separa os dados e os rótulos

        # Converte cada x para tensor se ainda for numpy
        if not isinstance(xs, torch.Tensor):
            xs = torch.from_numpy(xs).to(torch.float32)
        xs = torch.stack(xs).unsqueeze(2)  # (B, 6, 1, 60)

        # Converte ys para tensor também
        ys = torch.tensor(ys)
        return xs, ys

    return DataLoader(
        train_loader.dataset,
        batch_size=train_loader.batch_size,
        shuffle=getattr(train_loader, 'shuffle', True),
        num_workers=train_loader.num_workers,
        pin_memory=train_loader.pin_memory,
        drop_last=train_loader.drop_last,
        collate_fn=collate_fn_with_unsqueeze
    )


In [79]:
train_loader = reshape_train_loader(train_loader)

print(len(train_loader))
# Teste:
batch = next(iter(train_loader))
print(batch[0].shape)  # Esperado: torch.Size([64, 6, 1, 60])


475
torch.Size([64, 6, 1, 60])


In [63]:
gen_net = Generator(seq_len=60, channels=6, num_classes=6, latent_dim=100, data_embed_dim=10, 
                    label_embed_dim=10 ,depth=3, num_heads=5, 
                    forward_drop_rate=0.5, attn_drop_rate=0.5)

#print(gen_net)
dis_net = Discriminator(in_channels=6, patch_size=1, data_emb_size=50, label_emb_size=10, seq_length = 60, depth=3, n_classes=6)
#print(dis_net)

In [66]:
dis_net(batch[0])[0].shape, dis_net(batch[0])[1].shape

(torch.Size([64, 1]), torch.Size([64, 6]))

In [72]:
label = torch.randint(0, 5, (64,))
z = torch.FloatTensor(np.random.normal(0, 1, (64, 100)))

gen_net(z, label).shape

torch.Size([64, 6, 1, 60])